In [7]:
import random
import pandas as pd

In [8]:
mappings = {
  'San Francisco 49ers': '49ers',
  'Chicago Bears': 'Bears',
  'Cincinnati Bengals': 'Bengals',
  'Buffalo Bills': 'Bills',
  'Denver Broncos': 'Broncos',
  'Cleveland Browns': 'Browns',
  'Tampa Bay Buccaneers': 'Buccaneers',
  'Arizona Cardinals': 'Cardinals',
  'Los Angeles Chargers': 'Chargers',
  'Kansas City Chiefs': 'Chiefs',
  'Indianapolis Colts': 'Colts',
  'Washington Football Team': 'Commanders',
  'Washington Commanders': 'Commanders',
  'Dallas Cowboys': 'Cowboys',
  'Miami Dolphins': 'Dolphins',
  'Philadelphia Eagles': 'Eagles',
  'Atlanta Falcons': 'Falcons',
  'New York Giants': 'Giants',
  'Jacksonville Jaguars': 'Jaguars',
  'New York Jets': 'Jets',
  'Detroit Lions': 'Lions',
  'Green Bay Packers': 'Packers',
  'Carolina Panthers': 'Panthers',
  'New England Patriots': 'Patriots',
  'Las Vegas Raiders': 'Raiders',
  'Los Angeles Rams': 'Rams',
  'Baltimore Ravens': 'Ravens',
  'New Orleans Saints': 'Saints',
  'Seattle Seahawks': 'Seahawks', 
  'Pittsburgh Steelers': 'Steelers',
  'Houston Texans': 'Texans',
  'Tennessee Titans': 'Titans',
  'Minnesota Vikings': 'Vikings',
}

teams = list(set(mappings.values()))

In [18]:
def generate_team_power_ratings(random_range = (-15, 15)):
  power_rating = random.randint(random_range[0], random_range[1])
  power_rating += random.random()
  return { team: power_rating for team in teams }

home_power_ratings = generate_team_power_ratings()
away_power_ratings = generate_team_power_ratings()

In [16]:
df = pd.read_csv('../../data/nfl/pfr-2021-games-combo.csv', index_col=0)

df = df[df['week'].str.isdigit()]

df['week'] = df['week'].astype(str)
df['home'] = df['home'].astype(str).map(lambda a: mappings[a])
df['home_pts'] = df['home_pts'].astype(int)
df['away'] = df['away'].astype(str).map(lambda a: mappings[a])
df['away_pts'] = df['away_pts'].astype(int)
df['margin'] = df.apply(lambda a: a['home_pts'] - a['away_pts'], axis=1)

In [113]:
def tweak_power_ratings(power_ratings, random_range = (-1, 1), k_range = None):
   delta_power_ratings = power_ratings.copy()

   teams = list(delta_power_ratings.keys())

   if k_range is not None:
      k = random.randint(k_range[0], k_range[1])
   else:
      k = random.randint(2, len(teams))
   
   for team in random.sample(teams, k):
      delta_power_ratings[team] += random.randint(random_range[0], random_range[1])
      delta_power_ratings[team] += random.random()

   return delta_power_ratings

tweak_power_ratings(home_power_ratings)

{'Cardinals': 1.7635456797960343,
 'Texans': -7.0883432544860625,
 'Bengals': 2.1554063588898416,
 'Rams': 3.867736248059341,
 'Seahawks': 6.759613077827628,
 'Cowboys': 14.015215912700143,
 'Chargers': 6.437087049870834,
 'Steelers': -1.645984574151448,
 'Ravens': 8.09137638819605,
 'Vikings': 6.0822796850110885,
 'Lions': -3.0625288139273863,
 'Saints': -4.6221485598297285,
 'Bills': 7.4373570557455935,
 'Jaguars': -9.96330994227373,
 'Dolphins': -1.2104734599213205,
 'Raiders': -4.6800190011307965,
 'Jets': -11.370808291267027,
 'Commanders': -5.504908272948102,
 'Patriots': 4.670011763744691,
 'Falcons': -8.023375257599092,
 'Titans': 8.025192922055211,
 'Browns': -1.505533946073749,
 'Giants': -5.77435675170853,
 'Panthers': -9.877864520410267,
 'Buccaneers': 20.69758444521811,
 'Chiefs': 11.234903049005803,
 '49ers': 4.729781221371203,
 'Eagles': -0.6976201924051977,
 'Packers': 11.179412329987349,
 'Broncos': 8.56849464754345,
 'Colts': -1.7787569654972892,
 'Bears': 1.032993398

In [78]:
def evaluate(df, home_power_ratings, away_power_ratings, edge = 3):
  def prediction(row):
    return row['home_rating_adj'] - row['away_rating_adj']

  df_pred = df.copy()

  home_edge = .5 * edge
  away_edge = -home_edge

  df_pred['home_rating'] = df_pred['home'].map(home_power_ratings)
  df_pred['home_rating_adj'] = home_edge + df_pred['home_rating']
  df_pred['away_rating'] = df_pred['away'].map(away_power_ratings)
  df_pred['away_rating_adj'] = -away_edge + df_pred['away_rating']
  df_pred['prediction'] = df_pred.apply(prediction, axis=1)
  df_pred['error'] = df_pred.apply(lambda a: a['margin'] - a['prediction'], axis=1)
  df_pred['sq_error'] = df_pred['error'].map(lambda a: a * a)
  df_pred['abs_error'] = df_pred['error'].map(lambda a: abs(a))

  columns = [
    'week',
    'home',
    'home_pts',
    'home_rating',
    'home_rating_adj',
    'away',
    'away_pts',
    'away_rating',
    'away_rating_adj',
    'margin',
    'prediction',
    'error',
    'sq_error',
    'abs_error',
  ]

  return df_pred[columns]

df_pred = evaluate(df, home_power_ratings, away_power_ratings)

print('sq error:', df_pred['sq_error'].sum())
print('abs error:', df_pred['abs_error'].sum())
print()

df_pred.head()

sq error: 48223.067281039264



,week,home,home_pts,home_rating,home_rating_adj,away,away_pts,away_rating,away_rating_adj,prediction,error,sq_error,abs_error
0,1,Buccaneers,31,16.004210,17.504210,Cowboys,29,10.206490,11.706490,5.797721,-3.797721,14.422684,3.797721
1,1,Falcons,6,-6.550256,-5.050256,Eagles,32,1.691250,3.191250,-8.241506,-17.758494,315.364115,17.758494
2,1,Bills,16,16.900394,18.400394,Steelers,23,2.123089,3.623089,14.777305,-21.777305,474.251012,21.777305
3,1,Panthers,19,-4.836563,-3.336563,Jets,14,-12.511374,-11.011374,7.674811,-2.674811,7.154612,2.674811
4,1,Bengals,27,-4.817072,-3.317072,Vikings,24,-2.899676,-1.399676,-1.917396,4.917396,24.180780,4.917396


In [79]:
home_power_ratings = generate_team_power_ratings()
away_power_ratings = generate_team_power_ratings()

error_col = 'abs_error'

df_eval = evaluate(df, home_power_ratings, away_power_ratings)
global_error = df_eval[error_col].sum()

In [193]:
passes = 100
total_iterations_allowed_in_pass = 250
random_range = (-3, 3)
k_range = (2, 4)

print(f'global error @ 0:', global_error)

for i in range(1, passes):

  tse = global_error

  current_iteration = 1
  while current_iteration <= total_iterations_allowed_in_pass:
    delta_home_power_ratings = tweak_power_ratings(home_power_ratings, random_range, k_range)
    df_eval = evaluate(df, delta_home_power_ratings, away_power_ratings)
    error = df_eval[error_col].sum()
    if global_error > error:
      global_error = error
      home_power_ratings = delta_home_power_ratings

    delta_away_power_ratings = tweak_power_ratings(away_power_ratings, random_range, k_range)
    df_eval = evaluate(df, home_power_ratings, delta_away_power_ratings)
    error = df_eval[error_col].sum()
    if global_error > error:
      global_error = error
      away_power_ratings = delta_away_power_ratings

    current_iteration += 1

  if tse != global_error:
    print(f'global error @ {i}:', global_error)
  


global error @ 0: 2369.009834085744
global error @ 1: 2368.950289814581
global error @ 6: 2368.9242256279367
global error @ 7: 2368.9198717550685
global error @ 8: 2368.919871755068
global error @ 15: 2368.849830848081
global error @ 17: 2368.843011307217
global error @ 19: 2368.6144580059267
global error @ 20: 2368.6144580059263
global error @ 24: 2368.5250330749614
global error @ 27: 2368.29384290734
global error @ 29: 2368.1350711580258
global error @ 34: 2368.1286468663047
global error @ 37: 2368.0054548359144
global error @ 41: 2367.956349625201
global error @ 42: 2367.9563496252003
global error @ 46: 2367.9490679513574
global error @ 47: 2367.9326073440707
global error @ 48: 2367.8808722774747
global error @ 53: 2367.7381910984386
global error @ 56: 2367.6969637202305
global error @ 57: 2367.69696372023
global error @ 58: 2367.6158219826248
global error @ 75: 2367.615514237221
global error @ 82: 2367.5824572009797
global error @ 83: 2367.5824572009788
global error @ 84: 2367.5381

In [191]:
print('sq error:', global_error)
evaluate(df, home_power_ratings, away_power_ratings).head()

sq error: 2369.009834085744


,week,home,home_pts,home_rating,home_rating_adj,away,away_pts,away_rating,away_rating_adj,prediction,error,sq_error,abs_error
0,1,Buccaneers,31,19.963626,21.463626,Cowboys,29,9.562031,11.062031,10.401595,-8.401595,70.586797,8.401595
1,1,Falcons,6,-8.023375,-6.523375,Eagles,32,-1.228572,0.271428,-6.794803,-19.205197,368.839591,19.205197
2,1,Bills,16,7.281100,8.781100,Steelers,23,2.012037,3.512037,5.269063,-12.269063,150.529902,12.269063
3,1,Panthers,19,-9.938130,-8.438130,Jets,14,-14.903706,-13.403706,4.965576,0.034424,0.001185,0.034424
4,1,Bengals,27,2.268543,3.768543,Vikings,24,-0.716110,0.783890,2.984653,0.015347,0.000236,0.015347


In [192]:
sorted(home_power_ratings.items(), key=lambda a: a[1], reverse=True)

[('Buccaneers', 19.963626317120557),
 ('Cowboys', 14.941358931505608),
 ('Packers', 11.971832783523968),
 ('Chiefs', 9.808836313463582),
 ('Titans', 8.645819147262578),
 ('Ravens', 8.01704281091559),
 ('Bills', 7.28109968881705),
 ('Vikings', 7.231868193499502),
 ('Broncos', 7.039422062860454),
 ('Chargers', 6.592396043502086),
 ('Patriots', 4.670011763744691),
 ('49ers', 4.596697100237252),
 ('Seahawks', 4.238495631307929),
 ('Rams', 3.243164866430501),
 ('Bears', 3.228221968083998),
 ('Bengals', 2.2685426442236447),
 ('Cardinals', 1.807569366907158),
 ('Saints', -0.44478938121672795),
 ('Browns', -0.6284384784835987),
 ('Eagles', -0.6976201924051977),
 ('Colts', -1.553893262010817),
 ('Steelers', -1.645984574151448),
 ('Dolphins', -2.0671646907101398),
 ('Lions', -3.969400614496848),
 ('Raiders', -4.662852412884095),
 ('Commanders', -5.068144898912186),
 ('Giants', -5.498536418413552),
 ('Texans', -7.0883432544860625),
 ('Falcons', -8.023375257599092),
 ('Panthers', -9.93812973157570

In [126]:
sorted(away_power_ratings.items(), key=lambda a: a[1], reverse=True)

[('Cardinals', 17.943286668322116),
 ('Bills', 16.641355507817888),
 ('Bengals', 11.923826979049883),
 ('Chiefs', 11.298838086100867),
 ('Cowboys', 9.56203143182117),
 ('Rams', 8.916060764560182),
 ('Patriots', 7.945280617150511),
 ('Colts', 7.860728089232943),
 ('Saints', 6.67962703932802),
 ('49ers', 5.435229246476182),
 ('Buccaneers', 4.942276557887704),
 ('Packers', 4.8134067648027665),
 ('Browns', 2.9725551746740804),
 ('Steelers', 2.0120368930462496),
 ('Raiders', 1.3746180575886688),
 ('Vikings', -0.7161101262498396),
 ('Eagles', -1.2819098193514142),
 ('Titans', -1.9158040647846146),
 ('Chargers', -2.3956127061528036),
 ('Ravens', -2.641013533478342),
 ('Falcons', -2.9740952405420855),
 ('Broncos', -3.279557877630866),
 ('Bears', -3.627035778666332),
 ('Commanders', -3.932518343956742),
 ('Panthers', -4.181875665935271),
 ('Seahawks', -4.5919111784355335),
 ('Lions', -6.072001864304873),
 ('Dolphins', -7.674845214554322),
 ('Giants', -9.035324461973037),
 ('Texans', -10.8826983